In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

하이퍼파라미터 (Hyoeroarameter)
- epoch : 데이터셋을 반복하는 횟수 
- batch size : 매개변수가 갱신되기 전 신경망을 통해 전파된 데이터 샘플의 수 
- learning rate : 기본값 0.001 정도를 가짐

In [4]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # 예측(prediction)과 손실(loss) 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [6]:
loss_fn = nn.CrossEntropyLoss()# 손실 함수를 초기화한다.
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) # SGD를 사용

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.301613  [   64/60000]
loss: 2.297322  [ 6464/60000]
loss: 2.276867  [12864/60000]
loss: 2.265110  [19264/60000]
loss: 2.253167  [25664/60000]
loss: 2.220715  [32064/60000]
loss: 2.226585  [38464/60000]
loss: 2.198296  [44864/60000]
loss: 2.194863  [51264/60000]
loss: 2.154915  [57664/60000]
Test Error: 
 Accuracy: 42.4%, Avg loss: 2.157831 

Epoch 2
-------------------------------
loss: 2.169182  [   64/60000]
loss: 2.159382  [ 6464/60000]
loss: 2.106717  [12864/60000]
loss: 2.108350  [19264/60000]
loss: 2.063041  [25664/60000]
loss: 2.010669  [32064/60000]
loss: 2.022223  [38464/60000]
loss: 1.957480  [44864/60000]
loss: 1.955020  [51264/60000]
loss: 1.868292  [57664/60000]
Test Error: 
 Accuracy: 59.4%, Avg loss: 1.881582 

Epoch 3
-------------------------------
loss: 1.914289  [   64/60000]
loss: 1.880503  [ 6464/60000]
loss: 1.774581  [12864/60000]
loss: 1.799430  [19264/60000]
loss: 1.690997  [25664/60000]
loss: 1.651347  [32064/600